In [1]:
# !pip install pyngrok==4.1.1
!netstat -ano | findstr :4040
# !ngrok authtoken 2V9NV9WaROXzxtGB5GPFdLxoeLs_5F5EdtwiSPMHBubbUZdTE


In [2]:
import io
import pickle
import os.path
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.oauth2.credentials import Credentials
from flask import Flask, flash, request, redirect, url_for, jsonify, send_file, make_response,session
from flask_cors import CORS, cross_origin
from flask_ngrok import run_with_ngrok
from roboflow import Roboflow
import urllib
import cv2
import base64
import numpy as np
import io
from PIL import Image
import os
from gtts import gTTS

In [3]:
class Drive_OCR:
    def __init__(self,filename) -> None:
        self.filename = filename
        self.SCOPES = ['https://www.googleapis.com/auth/drive']
        self.credentials = "./credentials.json"
        self.pickle = "./token.json"

    def main(self) -> str:
        """Shows basic usage of the Drive v3 API.
        Prints the names and ids of the first 10 files the user has access to.
        """
        creds = None
        # The file token.pickle stores the user's access and refresh tokens, and is
        # created automatically when the authorization flow completes for the first
        # time.
        if os.path.exists('token.json'):
            creds = Credentials.from_authorized_user_file('token.json', self.SCOPES)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', self.SCOPES)
                creds = flow.run_local_server(host='localhost', port=5501)
            # Save the credentials for the next run
            with open('token.json', 'w') as token:
                token.write(creds.to_json())

        service = build('drive', 'v3', credentials=creds)

        # For Uploading Image into Drive
        mime = 'application/vnd.google-apps.document'
        file_metadata = {'name': self.filename, 'mimeType': mime}
        file = service.files().create(
            body=file_metadata,
            media_body=MediaFileUpload(self.filename, mimetype=mime)
        ).execute()
        print('File ID: %s' % file.get('id'))

        # It will export drive image into Doc
        request = service.files().export_media(fileId=file.get('id'),mimeType="text/plain")

        # For Downloading Doc Image data by request
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print("Download %d%%." % int(status.progress() * 100))

        # It will delete file from drive base on ID
        service.files().delete(fileId=file.get('id')).execute()

        # It will print data into terminal
        output = fh.getvalue().decode()
        return output

In [ ]:
if not os.path.isdir("cropped_images"):
  os.mkdir("cropped_images")
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'}

app = Flask(__name__)
CORS(app, support_credentials=True)
app.config['CORS_HEADERS'] = 'Content-Type'

app.secret_key = 'my_secret_key'

rf = Roboflow(api_key="sMDLRU6MwJA4ONNM61TU")
project = rf.workspace("urduocr").project("line-detection-urduocr")
model = project.version(2).model
run_with_ngrok(app)

urdu_chars = ['ت', 'ڈ', '{', 'ل', 'د', "'", 'ڑ', 'ژ', 'ض', '۱', ';', 'ﷲ', '۰', 'ن', '’', '؛', 'أ', '+', '[', '۶', '۳', ')', 'ٹ', '؟', 'چ', '﷽', 'ص', ',', 'ﷺ', 'ظ', '>', '"', 'ء', '&', '<', 'ا', '۷', 'ق', '۸', 'ٍ', 'ف', '۔', 'ج', 'ھ', 'ے', '۹', 'َ', 'ؒ', 'ٌ', '%', '.', ' ', '\n', 'ک', '۵', 'خ', '\ue002', 'ۃ', '\ue004', '؏', 'ُ', 'ں', ':', '`', 'ش', '^', '\ue000', '\u0602', 'پ', 'ع', 'ی', '*', 'ّ', 'ً', '~', ']', 'ح', 'ز', 'غ', '۴', 'ْ', '\ue001', '،', '_', '#', '÷', 'ث', 'گ', 'آ', 'ر', '․', 'ٰ', 'ِ', '\\', 'ۓ', 'س', '?', 'و', '‘', 'ٴ', 'ٗ', 'ؐ', 'ۂ', '؍', '(', 'ب', 'م', 'é', '٦', 'ؓ', 'ہ', 'ٔ', 'ط', '$', 'ٓ', 'ة', '|', '}', '/', 'ئ', '=', 'ؤ', 'ٖ', '۲', '-', 'ذ', '!', '@']
dat = pd.read_csv("character_mappings.csv", index_col=0, encoding="utf-8")
dat = dat.to_dict()["UrduEquivalent"]
cs = pd.read_csv("char_number.csv", encoding="utf-8", index_col=0)
cs = cs.to_dict()["number"]
dat.update(cs)

@app.route('/', methods=['GET'])
def start_page():
  return "Hello World!"

@app.route('/detection', methods=['GET', 'POST'])
def detect_lines():
    if request.method == 'POST':
        file = request.get_json()
        f = urllib.request.urlopen(file["file"])
        myfile = f.read()
        if f.info()['content-type'].split("/")[0] == "image":
            myfile = np.array(Image.open(io.BytesIO(myfile)))
            cv2.imwrite("save.jpg", myfile)
            detect = model.predict("save.jpg", confidence=10, overlap=30)
            data = detect.json()
            data['predictions'].sort(key=lambda x: x['y'])
            detect.save("save_predict.jpg")
            # Get the image bytes
            image_bytes = open('save_predict.jpg', 'rb').read()
            # Encode the image bytes to base64
            image_base64 = base64.b64encode(image_bytes).decode()
            os.remove("save_predict.jpg")
            return jsonify({"predicted_image": image_base64, "is_image": True, "my_data": data})


@app.route('/extraction', methods=['GET', 'POST'])
def extract_text():
    if request.method == 'POST':
        image = cv2.imread("save.jpg")
        count = 0
        data = request.get_json()["my_data"]
        generated_texts = []
        actual_generated_text = ""
        for i in data["predictions"]:
            if i['class'] == "Line":
                count +=1
                height = i['height']
                width = i['width']
                x = i['x']
                y = i['y']

                # Padded ROI
                roi_x = int(x - (width + 30)/ 2)
                roi_y = int(y - (height + 30)/ 2)
                roi_width = int(width + 30)
                roi_height = int(height + 30)

                # Non-Padded ROI
                # roi_x = int(x - (width)/ 2)
                # roi_y = int(y - (height)/ 2)
                # roi_width = int(width)
                # roi_height = int(height)


                roi = image[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]
                path = f"./cropped_images/cropped{count}.jpg"
#                 cropImage_list.append(path)
                cv2.imwrite(path, roi)
                count+=1

#                 pixel_values_test1 = processor.feature_extractor(roi, return_tensors="pt").pixel_values
#                 generated_ids_test1 = loaded_model.generate(pixel_values_test1)
#                 generated_text_test1 = processor.batch_decode(generated_ids_test1, skip_special_tokens=True)[0]
#                 generated_texts.append(generated_text_test1)
                while True:
                  try:
                    drive = Drive_OCR(path)
                    output = drive.main()
                    generated_texts.append(output.replace("\ufeff________________\r\n\r\n", ""))
                    os.remove(path)
                    break
                  except:
                    pass
        if len(generated_texts)>0:
            actual_generated_text = "\n".join(generated_texts)
        tts = gTTS(text=actual_generated_text, lang="ur")
        tts.save(f'./audio_lines.mp3')
        audio_bytes = open('./audio_lines.mp3', 'rb').read()
        audio_base64 = base64.b64encode(audio_bytes).decode('UTF-8')
        session['is_image'] = False
        os.remove("save.jpg")
        os.remove('./audio_lines.mp3')
        return jsonify({"generated_text": actual_generated_text, "audio": audio_base64 })

@app.route('/extract', methods=['GET', 'POST'])
def extract_complete_page():
    if request.method == 'POST':
        file = request.get_json()
        f = urllib.request.urlopen(file["file"])
        myfile = f.read()
        if f.info()['content-type'].split("/")[0] == "image":
            myfile = np.array(Image.open(io.BytesIO(myfile)))
            cv2.imwrite("save.jpg", myfile)
            drive = Drive_OCR("save.jpg")
            output = drive.main()
            output = output.replace("\ufeff________________\r\n\r\n", "")
            output_str = ""
            for char in output:
                if char not in urdu_chars:
                    if str(char).upper() in dat.keys():
                        output_str += dat[str(char).upper()]
                    else:
                        if not  output_str.endswith(" ") and output_str!="":
                            output_str += " "
                else:
                    output_str += char
                    
            tts = gTTS(text=output_str, lang="ur")
            tts.save(f'./audio_lines.mp3')
            audio_bytes = open('./audio_lines.mp3', 'rb').read()
            audio_base64 = base64.b64encode(audio_bytes).decode('UTF-8')
            image_bytes = open('save.jpg', 'rb').read()
            # Encode the image bytes to base64
            image_base64 = base64.b64encode(image_bytes).decode()
            os.remove("save.jpg")
            os.remove('./audio_lines.mp3')
            return jsonify({"generated_text": output_str, "audio": audio_base64, "image": image_base64 })

app.run()

loading Roboflow workspace...
loading Roboflow project...
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


 * Running on http://22a6-2400-adc1-401-1a00-f11e-bb3a-d136-e743.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Sep/2023 01:29:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2023 01:29:40] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Sep/2023 01:29:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2023 01:31:20] "OPTIONS /extract HTTP/1.1" 200 -


File ID: 1ydv1IOkCcgBCJ2CW6XtQGrS6wpBeVycc6wbHP9MWmqE
Download 100%.


127.0.0.1 - - [12/Sep/2023 01:32:12] "POST /extract HTTP/1.1" 200 -


In [16]:
ur = ['ت', 'ڈ', '{', 'ل', 'د', "'", 'ڑ', 'ژ', 'ض', '۱', ';', 'ﷲ', '۰', 'ن', '’', '؛', 'أ', '+', '[', '۶', '۳', ')', 'ٹ', '؟', 'چ', '4', '﷽', 'ص', '2', ',', 'ﷺ', 'ظ', '>', '"', 'ء', '&', '<', 'ا', '۷', 'ق', '۸', 'ٍ', 'ف', '۔', '8', 'ج', 'ھ', 'ے', '۹', 'َ', 'ؒ', 'ٌ', '%', '.', ' ', '\n', 'ک', '۵', 'خ', '\ue002', 'ۃ', '\ue004', '؏', '3', 'ُ', 'ں', ':', '`', 'ش', '^', '\ue000', '\u0602', 'پ', 'ع', 'ی', '*', 'ّ', 'ً', '~', '6', ']', 'ح', 'ز', 'غ', '۴', 'ْ', '\ue001', '9', '،', '_', '#', '÷', 'ث', 'گ', '5', 'آ', 'ر', '1', '․', 'ٰ', 'ِ', '\\', 'ۓ', 'س', '?', '0', 'و', '‘', 'ٴ', 'ٗ', 'ؐ', 'ۂ', '؍', '(', 'ب', 'م', 'é', '٦', 'ؓ', 'ہ', 'ٔ', 'ط', '$', 'ٓ', 'ة', '|', '}', '/', 'ئ', '=', 'ؤ', 'ٖ', '۲', '-', 'ذ', '!', '7', '@']
for i in range(10):
    if str(i) in ur:
        ur.remove(str(i))

In [17]:
len(ur)

128

In [19]:
print(ur)

['ت', 'ڈ', '{', 'ل', 'د', "'", 'ڑ', 'ژ', 'ض', '۱', ';', 'ﷲ', '۰', 'ن', '’', '؛', 'أ', '+', '[', '۶', '۳', ')', 'ٹ', '؟', 'چ', '﷽', 'ص', ',', 'ﷺ', 'ظ', '>', '"', 'ء', '&', '<', 'ا', '۷', 'ق', '۸', 'ٍ', 'ف', '۔', 'ج', 'ھ', 'ے', '۹', 'َ', 'ؒ', 'ٌ', '%', '.', ' ', '\n', 'ک', '۵', 'خ', '\ue002', 'ۃ', '\ue004', '؏', 'ُ', 'ں', ':', '`', 'ش', '^', '\ue000', '\u0602', 'پ', 'ع', 'ی', '*', 'ّ', 'ً', '~', ']', 'ح', 'ز', 'غ', '۴', 'ْ', '\ue001', '،', '_', '#', '÷', 'ث', 'گ', 'آ', 'ر', '․', 'ٰ', 'ِ', '\\', 'ۓ', 'س', '?', 'و', '‘', 'ٴ', 'ٗ', 'ؐ', 'ۂ', '؍', '(', 'ب', 'م', 'é', '٦', 'ؓ', 'ہ', 'ٔ', 'ط', '$', 'ٓ', 'ة', '|', '}', '/', 'ئ', '=', 'ؤ', 'ٖ', '۲', '-', 'ذ', '!', '@']
